In [11]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime
import requests

In [367]:
class HelperFunctions():
    def getYears(self,quarters): #go through each file name and find the year of filing
        return [int(re.findall("\d{4}",quarter)[0]) for quarter in quarters]
    
    def getUniqueYears(self,quarters):
        return set(self.getYears(quarters))
    
    def getPossibleFilingsFromYears(self, years):
        return [q + " " + str(year) for year in years for q in ["Q1","Q2","Q3","Q4","Year Ended"]]
    
    def getExcelSheetsPostDate(self,Excel):
        years = range(self.fromDate,self.endDate+1)
        names = self.getPossibleFilingsFromYears(years)
        #names = ["Year Ended "+str(self.fromDate-1)]+names
        files = [filing for name in names for filing in Excel.keys() if name in filing] 
        Excel = {file:Excel[file] for file in files}
        return Excel
    
    def reorderQuarters(self,compiledStatement):
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        cols = self.getPossibleFilingsFromYears(years)
        cols = [col for col in cols if col in quarters]
        compiledStatement = compiledStatement[["label"] + cols]
        return compiledStatement
        
    def getQuarters(self, compiledStatement):
        return list(compiledStatement.columns[1:])
    
    
    def cleanup_label(self, label):
        new_label = label.lower()
        new_label = re.sub("\([\w\W]+?\)","",new_label)
        new_label = re.sub(" \$|\s*\d+\,*", "", new_label) 
        new_label = new_label.replace("gain","loss").replace("decrease","increase").replace("  "," ").replace("—"," - ").replace("/","").strip()
        return new_label
    
    
    def addData(self, Excel):
        compiledStatement = pd.DataFrame(columns=["label"])
        excel = list(Excel.items())
        excel.reverse()
        for quarter,sheet in excel:
            quarter =  re.sub("\s\([\w\W]+?\)","",quarter) #remove the parenthesis ie. (2021) from the quarter
            sheet.columns = ["label",quarter]
            
            sheet = sheet.dropna(subset="label", axis=0, how="any")
            sheet = sheet[(~sheet[quarter].astype("string").str.contains("[a-zA-Z]",regex=True))|sheet[quarter].isnull()]
            
            sheet.label = sheet.label.apply(self.cleanup_label)
            
            sheeti = statementi = 0
            newStatement = pd.DataFrame()
            #merge the current finalstatement and new sheet into one statement 
            #go through each row of the two statements and keep track of the indexes of each statement
            #continue until one of either the compiled statement or sheet ends
            while sheeti < len(sheet) and statementi < len(compiledStatement):
                sheet_label = sheet.iloc[sheeti].label
                statement_label = compiledStatement.iloc[statementi].label
                newRowStatement = compiledStatement.iloc[statementi:statementi+1].reset_index(drop=True) #grab the row and drop the index so that they merge
                newRowSheet = sheet.iloc[sheeti:sheeti+1].reset_index(drop=True)

                #check if the labels are the same 
                if statement_label == sheet_label:
                    #if labels are the same then create a new row by concating the rows
                    newRowSheet = newRowSheet.drop(columns="label")
                    newRow = pd.concat([newRowStatement,newRowSheet],axis=1)
                    newRow["value"] = sheeti
                    newStatement = pd.concat([newStatement, newRow], ignore_index=True)
                    
                    #move on to the next row
                    sheeti+=1
                    statementi+=1
                    
                else:
                    maxi = min(sheeti+5,len(sheet))
                    
                    #next, before I append the final statement label, I want to check if the statement label corresponds to the next label for the sheet
                    #if it corresponds, then I will move the sheet up instead
                    if statement_label in list(sheet.iloc[sheeti:maxi].label): 
                        newRowSheet["value"] = sheeti
                        newStatement = pd.concat([newStatement,newRowSheet]) #concat the sheet row to the new statement
                        sheeti += 1
                        continue
                    
                    #if the two labels don't match, default is to add the row of the final statment
                    newStatement = pd.concat([newStatement,newRowStatement])
                    
                    statementi += 1
                    
                newStatement = newStatement.reset_index(drop=True)
                newStatement.index = newStatement.index+1
        
            #if either the finalstatement or the new sheet hasn't finished, then add the end of the statement to the new one
            sheet["value"] = sheet.index
            newStatement = pd.concat([newStatement,sheet.iloc[sheeti:len(sheet)]])
            newStatement = pd.concat([newStatement,compiledStatement.iloc[statementi:len(compiledStatement)]])

            compiledStatement = newStatement
        
        compiledStatement = compiledStatement.sort_values(by="value")#.drop(columns="value")
        compiledStatement = compiledStatement.reset_index(drop=True)
        return compiledStatement
    

In [368]:
class CompileIncomeStatement(HelperFunctions):
    def __init__(self,ticker,fromDate, endDate=2022):
        self.ticker = ticker
        self.fromDate = fromDate
        self.endDate = endDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.input = os.path.join(self.path, f"Income Statements All-{self.ticker}.xlsx")
        self.Excel = self.readExcel()
        self.Excel = self.getExcelSheetsPostDate(self.Excel)
        
        self.compiledStatement = self.addData(self.Excel)
        self.compiledStatement = self.performIncomeMath(self.compiledStatement)
        self.compiledStatement = self.reorderQuarters(self.compiledStatement)
    
    def readExcel(self):
        Excel = pd.read_excel(self.input,sheet_name=None)
        return Excel
    
    
    
    def performIncomeMath(self,compiledStatement):
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        
        for year in years:
            year = str(year)
            if "Year Ended "+year in quarters and "Q3 "+year in quarters: 
                compiledStatement["Q4 "+year] = compiledStatement["Year Ended "+year] - compiledStatement["Q3 "+year]
                compiledStatement.loc[compiledStatement.label.str.startswith("weighted-average"),"Q4 "+year] = compiledStatement.loc[compiledStatement.label.str.startswith("weighted-average"),"Year Ended "+year]
        return compiledStatement

            

In [369]:
class Income:
    def __init__(self,ticker,fromDate):
        self.ticker = ticker
        self.fromDate = fromDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.output = os.path.join(self.path, f"Compiled Income Statement-{self.ticker}.xlsx")
        
        self.createSeparateStatements()
        self.createStatement()
        
        self.writeExcel()
        
    def createSeparateStatements(self):
        self.separateCompiledStatement = pd.DataFrame()
        for year in range(self.fromDate,2023):
            statement = CompileIncomeStatement(self.ticker, year,year)
            self.separateCompiledStatement = pd.concat([self.separateCompiledStatement,statement.compiledStatement],axis=1)
    
    def createStatement(self):
        self.compiledStatement = CompileIncomeStatement(self.ticker, self.fromDate).compiledStatement
        
    def writeExcel(self):
        Excel = pd.ExcelWriter(self.output)
        self.compiledStatement.to_excel(Excel, index=False, sheet_name="Compiled Statement")
        self.separateCompiledStatement.to_excel(Excel, index=False, sheet_name="Separately Compiled Statement")
        
        Excel.save()
         
    

In [370]:
income = Income("MSFT",2017)
income.compiledStatement



IndexError: list index out of range

In [361]:
class CompileCashflowStatement(HelperFunctions):
    def __init__(self,ticker,fromDate,endDate=2022):
        self.ticker = ticker
        self.fromDate = fromDate
        self.endDate = endDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.input = os.path.join(self.path, f"Cashflow Statements All-{self.ticker}.xlsx")
        self.output = os.path.join(self.path, f"Compiled Cashflow Statement-{self.ticker}.xlsx")
        
        
        self.Excel = self.readExcel()
        self.Excel = self.getExcelSheetsPostDate(self.Excel)
        
        self.compiledStatement = self.addData(self.Excel)

        #self.compiledStatement = self.reorderQuarters(self.compiledStatement)

        
    def readExcel(self):
        Excel = pd.read_excel(self.input,sheet_name=None)
        return Excel
    
    
   
            

In [362]:
CompileCashflowStatement("GPS",2022).compiledStatement

,label,Year Ended 2022,value,Q3 2022,Q2 2022,Q1 2022
0,cash flows from operating activities:,NaN,0.0,NaN,NaN,NaN
1,net income,256.0,1.0,272.0,424.0,166.0
2,adjustments to reconcile net income to net cas...,NaN,2.0,NaN,NaN,NaN
3,depreciation and amortization,504.0,3.0,372.0,244.0,120.0
4,share-based compensation,139.0,4.0,97.0,72.0,36.0
...,...,...,...,...,...,...
71,"cash, cash equivalents, and restricted cash en...",902.0,48.0,NaN,NaN,NaN
72,cash flow non cash investing disclosure [abstr...,NaN,49.0,NaN,NaN,NaN
73,purchases of property and equipment not yet pa...,124.0,50.0,NaN,NaN,NaN
74,cash paid for interest during the period,180.0,52.0,NaN,NaN,NaN


In [355]:
class Cashflow:
    def __init__(self,ticker,fromDate):
        self.ticker = ticker
        self.fromDate = fromDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.output = os.path.join(self.path, f"Compiled Cashflow Statement-{self.ticker}.xlsx")
        
        self.createSeparateStatements()
        self.createStatement()
        
        self.writeExcel()
        
    def createSeparateStatements(self):
        self.separateCompiledStatement = pd.DataFrame()
        for year in range(self.fromDate,2023):
            statement = CompileCashflowStatement(self.ticker, year,year)
            self.separateCompiledStatement = pd.concat([self.separateCompiledStatement,statement.compiledStatement],axis=1)
    
    def createStatement(self):
        self.compiledStatement = CompileCashflowStatement(self.ticker, self.fromDate).compiledStatement
        
    def writeExcel(self):
        Excel = pd.ExcelWriter(self.output)
        self.compiledStatement.to_excel(Excel, index=False, sheet_name="Compiled Statement")
        self.separateCompiledStatement.to_excel(Excel, index=False, sheet_name="Separately Compiled Statement")
        
        Excel.save()
         
    

In [356]:
cashflow = Cashflow("GPS",2019)
cashflow.compiledStatement



,label,Q1 2021,Year Ended 2022,Q3 2022,Q2 2022,Q1 2022,Year Ended 2021,Q3 2021,Q2 2021,Year Ended 2020,Q3 2020,Q2 2020,Q1 2020,Year Ended 2019,Q3 2019,Q2 2019,Q1 2019
0,other asset impairment charges,124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cash flows from operating activities:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"operating lease, impairment loss",360.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,net income,-932.0,256.0,272.0,424.0,166.0,-665.0,-899.0,-994.0,351.0,535.0,395,227,1003.0,727.0,461.0,164.0
4,adjustments to reconcile net income to net cas...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,cash flow non cash investing disclosure [abstr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,capital expenditures incurred but not yet paid,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,"interest paid, including capitalized interest,...",NaN,NaN,NaN,NaN,NaN,145.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,income taxes paid,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
class CompileBalanceStatement(HelperFunctions):
    def __init__(self,ticker,fromDate,endDate=2022):
        self.ticker = ticker
        self.fromDate = fromDate
        self.endDate = endDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.input = os.path.join(self.path, f"Balance Statements All-{self.ticker}.xlsx")
        self.output = os.path.join(self.path, f"Compiled Balance Statement-{self.ticker}.xlsx")
        
        
        self.Excel = self.readExcel()
        self.Excel = self.getExcelSheetsPostDate(self.Excel)
        
        self.compiledStatement = self.addData(self.Excel)

        self.compiledStatement = self.reorderQuarters(self.compiledStatement)

        
    def readExcel(self):
        Excel = pd.read_excel(self.input,sheet_name=None)
        return Excel
    
    
   
            

In [213]:
class Balance:
    def __init__(self,ticker,fromDate):
        self.ticker = ticker
        self.fromDate = fromDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.output = os.path.join(self.path, f"Compiled Balance Statement-{self.ticker}.xlsx")
        
        self.createSeparateStatements()
        self.createStatement()
        
        self.writeExcel()
        
    def createSeparateStatements(self):
        self.separateCompiledStatement = pd.DataFrame()
        for year in range(self.fromDate,2023):
            statement = CompileBalanceStatement(self.ticker, year,year)
            self.separateCompiledStatement = pd.concat([self.separateCompiledStatement,statement.compiledStatement],axis=1)
    
    def createStatement(self):
        self.compiledStatement = CompileBalanceStatement(self.ticker, self.fromDate).compiledStatement
        
    def writeExcel(self):
        Excel = pd.ExcelWriter(self.output)
        self.compiledStatement.to_excel(Excel, index=False, sheet_name="Compiled Statement")
        self.separateCompiledStatement.to_excel(Excel, index=False, sheet_name="Separately Compiled Statement")
        
        Excel.save()
         
    

In [218]:
balance = Balance("MSFT",2019)
balance.compiledStatement



Year Ended 2018
Q1 2019 (2018)
Q2 2019 (2018)
Q3 2019
Year Ended 2019
Year Ended 2019
Q1 2020 (2019)
Q2 2020 (2019)
Q3 2020
Year Ended 2020
Year Ended 2020
Q1 2021 (2020)
Q2 2021 (2020)
Q3 2021
Year Ended 2021
Year Ended 2021
Q1 2022 (2021)
Q2 2022 (2021)
Q3 2022
Year Ended 2022
Year Ended 2018
Q1 2019 (2018)
Q2 2019 (2018)
Q3 2019
Year Ended 2019
Q1 2020 (2019)
Q2 2020 (2019)
Q3 2020
Year Ended 2020
Q1 2021 (2020)
Q2 2021 (2020)
Q3 2021
Year Ended 2021
Q1 2022 (2021)
Q2 2022 (2021)
Q3 2022
Year Ended 2022


,label,Year Ended 2018,Q1 2019,Q2 2019,Q3 2019,Year Ended 2019,Q1 2020,Q2 2020,Q3 2020,Year Ended 2020,Q1 2021,Q2 2021,Q3 2021,Year Ended 2021,Q1 2022,Q2 2022,Q3 2022,Year Ended 2022
0,current assets:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,cash and cash equivalents,11946,15137,6638,11212,11356,13117,8864,11710,13576,17205,14432,13702,14224,19165,20604,12498,13931
2,short-term investments,121822,120743,121024,120406,122463,123519,125389,125916,122951,120772,117536,111705,116110,111450,104765,92195,90826
3,"total cash, cash equivalents, and short-term i...",133768,135880,127662,131618,133819,136636,134253,137626,136527,137977,131968,125407,130334,130615,125369,104693,104757
4,"accounts receivable, net of allowance for doub...",26481,17390,19680,19269,29524,19087,23525,22699,32011,22851,27312,26322,38043,27349,33520,32613,44261
5,inventories,2662,3614,1961,1951,2063,2622,1823,1644,1895,2705,1924,2245,2636,3411,3019,3296,3742
6,other,6751,7311,7571,7049,10146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,other current assets,NaN,NaN,NaN,NaN,NaN,7551,7473,8536,11482,13544,12769,11640,13393,12951,12280,13320,16924
8,total current assets,169662,164195,156874,159887,175552,165896,167074,170505,181915,177077,173973,165614,184406,174326,174188,153922,169684
9,"property and equipment, net of accumulated dep...",29460,31430,32717,33648,36477,38409,40522,41221,44151,47927,51737,54945,59715,63772,67214,70298,74398
